In [25]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @staticmethod
    def pallottoliere(max_nums):
        while True:
            yield rnd.randint(1, max_nums)

    @staticmethod
    def choice(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        def extraction():
            rnd.shuffle(whole_numbers)
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return [extraction() for _ in range(nums)]

    @staticmethod
    def sample(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        return rnd.sample(whole_numbers, k=nums)

    def int(self, nums, max_nums):
        combo = set()
        while len(combo) < nums:
            combo.add(next(self.pallottoliere(max_nums)))

        return combo

    @staticmethod
    def manySamples(backend):
        def inner(nums, max_nums):
            samples = [backend(nums, max_nums) for _ in range(max_nums)]
            rnd.shuffle(samples)

            return rnd.choice(samples)

        return inner

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        backend = self.manySamples(backend) if self.many else backend

        combo = backend(self.numbers, self.max_numbers)
        if self.extra:
            extra = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nNumeri Estratti:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(extra))  # type: ignore

In [26]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, numbers=6, extra=0)

In [27]:
superenalotto(backend='choice')

Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 11 18 30 68 83 87


In [28]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [29]:
eurojack()

Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 7 25 31 43 48
Superstar: 9 10


In [30]:
for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=False)
    print('backend:', eurojack.backend)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=True)
    print('backend:', eurojack.backend)

######
Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 10 38 39 45 48
Superstar: 5 11
backend: choice
######
Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 8 22 25 28 35
Superstar: 4 6
backend: int
######
Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 35 40 41 42 49
Superstar: 1 10
backend: sample

################# many ####################

######
Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 37 40 42 47 48
Superstar: 6 7
backend: choice
######
Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 6 13 16 25 39
Superstar: 1 7
backend: int
######
Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 5 21 29 34 45
Superstar: 2 7
backend: sample


In [31]:
superenalotto.extra = 1
superenalotto(backend='choice', many=False)

Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 14 19 24 37 59 64
Superstar: 88


In [32]:
superenalotto(backend='int')

Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 5 17 27 30 69 76
Superstar: 82


In [33]:
superenalotto.extra

1

In [34]:
superenalotto.many

True

In [35]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='int', many=False)

Estrazione del: 05/06/2023 16:57 
Numeri Estratti: 3 5 8 9 10 12 14 15 17 20
Superstar: 4


In [36]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'choice'

AttributeError: property 'backend' of 'Lotteria' object has no setter